In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from keras_tuner import HyperParameters

import autokeras as ak

from cerebro.nas.hphpmodel import HyperHyperModel

In [2]:
input_node = ak.StructuredDataInput()
output_node = ak.StructuredDataBlock(categorical_encoding=True)(input_node)
output_node = ak.ClassificationHead()(output_node)
am = HyperHyperModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=3
)

In [3]:
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .appName("Linear Regression Model") \
   .getOrCreate()

sc = spark.sparkContext

from cerebro.backend import SparkBackend
from cerebro.storage import LocalStore

backend = SparkBackend(spark_context=sc, num_workers=1)
store = LocalStore(prefix_path='/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/experiments')

am.resource_bind(
    backend=backend, 
    store=store,
    feature_columns=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'],
    label_columns=['Species']
)

21/11/19 22:31:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


CEREBRO => Time: 2021-11-19 22:31:02, Running 1 Workers


In [27]:
df = spark.read.csv("/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/Iris_clean.csv", header=True)
df = df.toPandas()

train=df.sample(frac=0.8,random_state=200) #random state is a seed value
test=df.drop(train.index)

df.head(10)
# train_df, test_df = df.randomSplit([0.8, 0.2])

[Row(SepalLengthCm='5.1', SepalWidthCm='3.5', PetalLengthCm='1.4', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='4.9', SepalWidthCm='3.0', PetalLengthCm='1.4', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='4.7', SepalWidthCm='3.2', PetalLengthCm='1.3', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='4.6', SepalWidthCm='3.1', PetalLengthCm='1.5', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='5.0', SepalWidthCm='3.6', PetalLengthCm='1.4', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='5.4', SepalWidthCm='3.9', PetalLengthCm='1.7', PetalWidthCm='0.4', Species='0'),
 Row(SepalLengthCm='4.6', SepalWidthCm='3.4', PetalLengthCm='1.4', PetalWidthCm='0.3', Species='0'),
 Row(SepalLengthCm='5.0', SepalWidthCm='3.4', PetalLengthCm='1.5', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='4.4', SepalWidthCm='2.9', PetalLengthCm='1.4', PetalWidthCm='0.2', Species='0'),
 Row(SepalLengthCm='4.9', SepalWidthCm='3.1', PetalLengthCm='1.5', PetalWidthCm='0.1', Spec

In [5]:
train_np = np.array(train)
x_train = train_np[:,:-1]
y_train = train_np[:,-1,np.newaxis]
print(x_train.shape)
print(y_train.shape)

(120, 4)
(120, 1)


In [6]:
cuz_hps = HyperParameters()
cuz_hps.Choice('learning_rate', values=[0.1,0.01])
cuz_hps.Choice('batch_size', values=[32,64,128])

am.tuner_bind(tuner="randomsearch", hyperparameters=cuz_hps)
am.test_tuner_space(x=x_train, y=y_train)

am.tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project ./test/oracle.json


2021-11-19 22:31:05.973877: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-19 22:31:05.974195: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Search space summary
Default search space size: 10
learning_rate (Choice)
{'default': 0.1, 'conditions': [], 'values': [0.1, 0.01], 'ordered': True}
batch_size (Choice)
{'default': 32, 'conditions': [], 'values': [32, 64, 128], 'ordered': True}
structured_data_block_1/normalize (Boolean)
{'default': False, 'conditions': []}
structured_data_block_1/dense_block_1/use_batchnorm (Boolean)
{'default': False, 'conditions': []}
structured_data_block_1/dense_block_1/num_layers (Choice)
{'default': 2, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
structured_data_block_1/dense_block_1/units_0 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512, 1024], 'ordered': True}
structured_data_block_1/dense_block_1/dropout (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.25, 0.5], 'ordered': True}
structured_data_block_1/dense_block_1/units_1 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64, 128, 256, 512, 1024], 'ordered': True}
classific

In [23]:
tuner = am.tuner

trials = tuner.oracle.create_trials(2)
for trial in trials:
    print(trial.hyperparameters.values)

{'learning_rate': 0.1, 'batch_size': 32, 'structured_data_block_1/normalize': False, 'structured_data_block_1/dense_block_1/use_batchnorm': False, 'structured_data_block_1/dense_block_1/num_layers': 3, 'structured_data_block_1/dense_block_1/units_0': 64, 'structured_data_block_1/dense_block_1/dropout': 0.25, 'structured_data_block_1/dense_block_1/units_1': 1024, 'classification_head_1/dropout': 0.5, 'optimizer': 'adam'}
{'learning_rate': 0.1, 'batch_size': 64, 'structured_data_block_1/normalize': False, 'structured_data_block_1/dense_block_1/use_batchnorm': True, 'structured_data_block_1/dense_block_1/num_layers': 1, 'structured_data_block_1/dense_block_1/units_0': 256, 'structured_data_block_1/dense_block_1/dropout': 0.5, 'structured_data_block_1/dense_block_1/units_1': 512, 'classification_head_1/dropout': 0.0, 'optimizer': 'sgd'}


In [19]:
kt_trial = tuner.oracle.create_trial(tuner.tuner_id)
kt_trial.hyperparameters.values

{'learning_rate': 0.01,
 'batch_size': 128,
 'structured_data_block_1/normalize': False,
 'structured_data_block_1/dense_block_1/use_batchnorm': True,
 'structured_data_block_1/dense_block_1/num_layers': 2,
 'structured_data_block_1/dense_block_1/units_0': 512,
 'structured_data_block_1/dense_block_1/dropout': 0.5,
 'structured_data_block_1/dense_block_1/units_1': 128,
 'classification_head_1/dropout': 0.0,
 'optimizer': 'adam_weight_decay'}

In [24]:
tuner.hypermodel._get_loss()

AttributeError: 'KerasHyperModel' object has no attribute '_get_loss'